# Introduction to uproot

The interface is minimal: open a file with `uproot.open` and extract objects with a dictionary-like interface. Let's open a NanoAOD file.

In [1]:
import uproot
tree = uproot.open("~/storage/data/nano-TTLHE-2017-09-04-lz4.root")["Events"]
tree

<TTree 'Events' len=55432 at 0x7b48d3df6950>

Incidentally, the one-liner above is not possible with PyROOT because ROOT's and Python's notions of object ownership conflict.

In [2]:
import ROOT
tree2 = ROOT.TFile("/home/pivarski/storage/data/nano-TTLHE-2017-09-04-lz4.root").Get("Events")
tree2

Welcome to JupyROOT 6.08/04


<ROOT.TTree object at 0x(nil)>

As with most Python modules, all of the class members and methods that don't start with underscore are public.

In [3]:
print(", ".join(x for x in dir(tree) if not x.startswith("_")))

Counter, allbranches, allbranchnames, allbranchtypes, array, arrays, arrowed, branch, branches, branchnames, branchtypes, classes, counter, iterator, lazyarray, lazyarrays, leaves, name, numentries, pandas, title


In [4]:
help(tree.array)

Help on method array in module uproot.tree:

array(self, branch, dtype=None, executor=None, block=True) method of uproot.tree.TTree instance
    Extracts a whole branch into a Numpy array.
    
    Individual branches from TTrees are typically small enough to fit into memory. If this is not your case, consider `tree.iterator(entries)` to load a given number of entries at a time.
    
    Arguments:
    
        * `branch` *(required)*
    
           Branch name to extract.
    
        * `dtype`
    
           If not `None`, cast the array into a given `dtype` (such as conversion to little endian).
           If an array object, fill that array instead of creating a new one (shape must match).
    
        * `executor` (same as in `TTree.arrays`)
    
          A `concurrent.futures.Executor` that would be used to parallelize the basket loading/decompression.
          If `None`, the process is serial.
    
        * `block`
    
          If `True` and parallel processing with an `e

In [5]:
tree.branchnames

['run',
 'luminosityBlock',
 'event',
 'nElectron',
 'Electron_deltaEtaSC',
 'Electron_dxy',
 'Electron_eta',
 'Electron_mass',
 'Electron_mvaSpring16GP',
 'Electron_mvaSpring16HZZ',
 'Electron_phi',
 'Electron_pt',
 'Electron_pterr',
 'Electron_r9',
 'Electron_sieie',
 'Electron_sip3d',
 'Electron_charge',
 'Electron_cutBased',
 'Electron_cutBased_HLTPreSel',
 'Electron_jet',
 'Electron_mvaSpring16GP_WP80',
 'Electron_mvaSpring16GP_WP90',
 'Electron_mvaSpring16HZZ_WPL',
 'Electron_pdgId',
 'nFatJet',
 'FatJet_area',
 'FatJet_btagCMVA',
 'FatJet_btagDeepB',
 'FatJet_btagDeepBB',
 'FatJet_eta',
 'FatJet_mass',
 'FatJet_mpruned',
 'FatJet_msoftdrop',
 'FatJet_phi',
 'FatJet_pt',
 'FatJet_tau1',
 'FatJet_tau2',
 'FatJet_tau3',
 'FatJet_subJet1',
 'FatJet_subJet2',
 'FatJet_subJet3',
 'nGenJet',
 'GenJet_eta',
 'GenJet_mass',
 'GenJet_phi',
 'GenJet_pt',
 'nGenPart',
 'GenPart_eta',
 'GenPart_phi',
 'GenPart_pt',
 'GenPart_mother',
 'GenPart_pdgId',
 'GenPart_status',
 'genWeight',
 'LHEWe

In [7]:
len(tree.array("Electron_pt"))

61788

In [8]:
tree.arrays(["Electron_pt", "Electron_eta", "Electron_phi"])

{'Electron_eta': array([-0.50317383, -0.22283936,  0.3380127 , ...,  0.16601562,
        -1.51855469,  2.38574219], dtype=float32),
 'Electron_phi': array([-1.97704196,  2.73374438, -2.23343873, ..., -1.10651565,
         1.69811368,  2.38514328], dtype=float32),
 'Electron_pt': array([ 21.26379776,   5.12322283,  14.14859962, ...,  16.72582626,
         18.65698242,  15.28109741], dtype=float32)}

The branch/dtype argument of methods like `arrays` can take a function from `TBranch` to `dtype/None` as an argument, providing a flexible way to select branches and possibly change their Numpy `dtype` on the fly.

In [9]:
tree.arrays(lambda branch: branch.dtype if branch.name.startswith("Electron_") else None)

{'Electron_charge': array([-1, -1,  1, ..., -1, -1,  1], dtype=int32),
 'Electron_cleanmask': array([1, 1, 1, ..., 1, 1, 1], dtype=uint8),
 'Electron_cutBased': array([0, 0, 0, ..., 0, 0, 0], dtype=int32),
 'Electron_cutBased_HLTPreSel': array([0, 0, 0, ..., 0, 0, 0], dtype=int32),
 'Electron_deltaEtaSC': array([ 0.00846107,  0.00699943,  0.0149112 , ...,  0.01897203,
         0.00869478,  0.02348849], dtype=float32),
 'Electron_dxy': array([-0.38358903, -0.00145535,  0.00341794, ..., -0.00323985,
         0.00780344,  0.01845314], dtype=float32),
 'Electron_eta': array([-0.50317383, -0.22283936,  0.3380127 , ...,  0.16601562,
        -1.51855469,  2.38574219], dtype=float32),
 'Electron_jet': array([3, 0, 1, ..., 4, 0, 3], dtype=int32),
 'Electron_mass': array([-0.00792694,  0.00127983,  0.00332069, ...,  0.00569153,
         0.01219177,  0.00673294], dtype=float32),
 'Electron_mvaSpring16GP': array([-0.99991018, -0.9973309 , -0.7685014 , ..., -0.99344712,
        -0.64150786, -0.9586

For instance, we can change all numbers from the "big endian" format ROOT stores them in to your machine's native byte order.

In [10]:
tree.arrays(lambda branch: branch.dtype.newbyteorder("=") if branch.name.startswith("Electron_") else None)

{'Electron_charge': array([-1, -1,  1, ..., -1, -1,  1], dtype=int32),
 'Electron_cleanmask': array([1, 1, 1, ..., 1, 1, 1], dtype=uint8),
 'Electron_cutBased': array([0, 0, 0, ..., 0, 0, 0], dtype=int32),
 'Electron_cutBased_HLTPreSel': array([0, 0, 0, ..., 0, 0, 0], dtype=int32),
 'Electron_deltaEtaSC': array([ 0.00846107,  0.00699943,  0.0149112 , ...,  0.01897203,
         0.00869478,  0.02348849], dtype=float32),
 'Electron_dxy': array([-0.38358903, -0.00145535,  0.00341794, ..., -0.00323985,
         0.00780344,  0.01845314], dtype=float32),
 'Electron_eta': array([-0.50317383, -0.22283936,  0.3380127 , ...,  0.16601562,
        -1.51855469,  2.38574219], dtype=float32),
 'Electron_jet': array([3, 0, 1, ..., 4, 0, 3], dtype=int32),
 'Electron_mass': array([-0.00792694,  0.00127983,  0.00332069, ...,  0.00569153,
         0.01219177,  0.00673294], dtype=float32),
 'Electron_mvaSpring16GP': array([-0.99991018, -0.9973309 , -0.7685014 , ..., -0.99344712,
        -0.64150786, -0.9586

If you already have an array, you can pass it in place of the `dtype` argument. This avoids unnecessary copies.

In [11]:
import numpy
electron_pt = numpy.zeros(tree.array("nElectron").sum(), dtype=numpy.float64)
id(electron_pt)

135552227028704

In [12]:
electron_pt

array([ 0.,  0.,  0., ...,  0.,  0.,  0.])

In [13]:
tree.array("Electron_pt", electron_pt)
id(electron_pt)

135552227028704

In [14]:
electron_pt

array([ 21.26379776,   5.12322283,  14.14859962, ...,  16.72582626,
        18.65698242,  15.28109741])

If the arrays are too large to read all at once, you can iterate over them.

In [20]:
for pt, eta, phi in tree.iterator(1000, ["Electron_pt", "Electron_eta", "Electron_phi"], outputtype=tuple):
    print("px = {}".format(pt.shape))

Or over a collection of files (like TChain).

In [21]:
for pt, eta, phi in uproot.iterator(1000, "~/storage/data/nano-TTLHE-2017-09-04-*.root", "Events", ["Electron_pt", "Electron_eta", "Electron_phi"], outputtype=tuple):
    print("px = {}".format(pt*numpy.cosh(eta)*numpy.sin(phi)))

uproot uses Python's `Executor` interface for parallelism. Parallel processing and caching are never implicit: you have to give it an object.

In [ ]:
import concurrent.futures
four_workers = concurrent.futures.ThreadPoolExecutor(4)

tree.arrays(["Electron_pt", "Electron_eta", "Electron_phi"], executor=four_workers)

In [ ]:
# returns immediately, before actually reading
arrays, errors = tree.arrays(["Electron_pt", "Electron_eta", "Electron_phi"],
                                                             executor=four_workers, block=False)

In [ ]:
# evaluate this iterator to wait for arrays to be filled and see if there were any errors
list(errors)

I'm adding connectors to other libraries, but I need your input about which are the most important.

In [22]:
df = tree.pandas.df(lambda branch: branch.dtype if branch.name.startswith("Electron_") else None)
df

Electron_deltaEtaSC  Electron_dxy  Electron_eta  Electron_mass  \
0                 0.008461     -0.383589     -0.503174      -0.007927   
1                 0.006999     -0.001455     -0.222839       0.001280   
2                 0.014911      0.003418      0.338013       0.003321   
3                 0.007591      0.002976      2.422363      -0.008163   
4                -0.038997     -0.000002      0.124268      -0.005939   
5                -0.031107     -0.006901     -1.384521       0.002663   
6                -0.034951      0.001633     -0.541992       0.004280   
7                 0.014235     -0.001128     -1.192627      -0.011436   
8                -0.003861     -0.001881     -1.731201       0.006336   
9                 0.001918     -0.003880     -1.718262       0.004139   
10               -0.045260     -0.000441     -0.426270      -0.012032   
11               -0.040411      0.006203      0.516113      -0.021637   
12               -0.057492     -0.001112      0.516479      -0.004463   
13               -0.016972     -0.000932      0.683105       0.001321   
14               -0.012513     -0.002123     -0.148987      -0.022003   
15                0.000457      0.132736     -0.927002      -0.012459   
16               -0.013742     -0.003523      1.873779       0.015076   
17               -0.000395     -0.036453     -0.799438      -0.006306   
18                0.003874      0.035192      0.284546       0.002708   
19                0.046427      0.000661      0.300964      -0.001128   
20                0.010962     -0.102307      1.255859       0.001590   
21                0.008554      1.112992      0.968628       0.002007   
22                0.016829      0.002468     -2.308105       0.041992   
23               -0.018588      0.120428      2.387207      -0.086975   
24               -0.034515     -0.000253      1.269043      -0.008530   
25               -0.003399     -0.001070      1.208984       0.019028   
26               -0.005649      0.000824      0.548218      -0.027313   
27               -0.006180     -0.000106     -0.726929      -0.003929   
28               -0.008857     -0.000241     -0.378540      -0.009483   
29               -0.007583     -0.001185     -2.219238      -0.006924   
...                    ...           ...           ...            ...   
61758             0.005525      0.001278     -0.688843      -0.011749   
61759             0.008323     -0.002139      1.068848       0.002634   
61760             0.005535      0.031408     -1.340332       0.004711   
61761             0.014676     -0.001824     -0.944946       0.039032   
61762             0.018566     -0.001466      0.278748       0.001227   
61763             0.021612      0.002759     -0.733643      -0.042877   
61764             0.024181     -0.000866     -0.309937       0.001649   
61765             0.025291      0.037461      0.220398      -0.001961   
61766            -0.043402      0.000531     -0.054619       0.029907   
61767            -0.023368     -0.000692      0.374023      -0.002155   
61768             0.000674      0.041194      1.052246      -0.004257   
61769            -0.001067      0.270533     -1.741455      -0.034058   
61770             0.009847     -0.002269      1.436523      -0.004135   
61771            -0.039920      0.006390     -0.824219      -0.006321   
61772            -0.004337      0.002111     -1.874512      -0.040283   
61773             0.048012      0.001560     -1.334961      -0.003910   
61774             0.068510      0.031867      1.902588      -0.004528   
61775             0.032835      0.000383     -1.410400      -0.005501   
61776             0.051053     -0.012421      0.502319      -0.001626   
61777            -0.024055     -0.001455     -1.032715      -0.004108   
61778             0.017687     -0.007056      2.115723       0.040588   
61779             0.035281     -0.492339      1.752930       0.010025   
61780            -0.002992     -0.000561     -0.764160       0.00

Now you can do that whole Pandas-analysis thing. See StackOverflow for help.

In [23]:
%matplotlib inline

px = (df.Electron_pt * numpy.cosh(df.Electron_eta) * numpy.sin(df.Electron_phi))

px.plot.hist(numpy.linspace(-100, 100, 200), edgecolor="none")

## Next steps

By now, you've probably noticed that we're limited to in-memory analytics and flat ntuples.

For a Pandas interface on a large set of files (out-of-memory analytics; what you've come to expect from ROOT's `TChain`), it could be interesting to try Blaze. Ask me about it if you're interested.

<table>
<tr style="background: white"><td><center><img src="http://blaze.pydata.org/images/blaze.png" width="200"></center></td></tr>
<tr><td style="font-size: 18pt">Blaze <a href="http://blaze.pydata.org/">blaze.pydata.org</a></td></tr>
</table>

For non-flat data (nested classes), let's move on to the next notebook.